# Video Processor

This is the video processor. It loads the model created previously and uses it to predict in real time if the animal on the current frame is a cow or not.<br/>

To do so, a multithreading environment is created. While the video is being processed with a ratio of 1 fps, the model is running on a parallel thread and receiving this frames. For each frame received, the model is predicting whether ther is a cow or not within the given frame.

In [1]:
from keras.models import model_from_json
import tensorflow as tf
import json
from PIL import Image
import cv2
import numpy as np
import os
import sys
import uuid

import threading

Using TensorFlow backend.


In [2]:
global model
global graph

with open("../Image_Recognizer/model.json", "r") as json_file:
    model = model_from_json(json_file.read())
    model.load_weights("../Image_Recognizer/model.h5")
    graph = tf.get_default_graph()
    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 50, 50, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        18496     
__________

In [3]:
frame = None

class Analysis_Thread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        while (not(frame is None)):
            self.Predict_frame(frame)

    def ResizeImage(self, image):
        return np.array(Image.fromarray(image, 'RGB').resize((50, 50)))

    def Get_frame_name(self, label):
        if label == 0:
            return "cow"
        elif label == 1:
            return "not cow"
    
    def Predict_frame(self, frame):
        with graph.as_default():
            score = model.predict(np.array([self.ResizeImage(frame)/255]), verbose=1)
            label_index = np.argmax(score)
            accuracy = np.max(score)
            print(str(count) + ": " + self.Get_frame_name(label_index) + " with accuracy = " + str(accuracy))
            cv2.imwrite("data/Processed/" + str(count) + ".jpg", frame)

In [5]:
count = 0
vidcap = cv2.VideoCapture("data/Raw/cow/videoplayback.mp4")
success, frame = vidcap.read()

keras_thread = Analysis_Thread()
keras_thread.start()

while success:
    vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
    success, frame = vidcap.read()
    count = count + 1

1/1 [==============================] - 0s 16ms/step
1: cow with accuracy = 0.6050858
1/1 [==============================] - 0s 16ms/step
4: not cow with accuracy = 0.9161762
1/1 [==============================] - 0s 7ms/step
6: cow with accuracy = 0.9404437
1/1 [==============================] - 0s 6ms/step
8: cow with accuracy = 0.7224814
1/1 [==============================] - 0s 14ms/step
10: cow with accuracy = 0.7224814
1/1 [==============================] - 0s 7ms/step
13: cow with accuracy = 0.9977563
1/1 [==============================] - 0s 7ms/step
14: cow with accuracy = 0.88294035
1/1 [==============================] - 0s 14ms/step
16: cow with accuracy = 0.9897625
1/1 [==============================] - 0s 7ms/step
18: cow with accuracy = 0.9113221
1/1 [==============================] - 0s 6ms/step
19: cow with accuracy = 0.8666692
1/1 [==============================] - 0s 16ms/step
23: cow with accuracy = 0.75401396
1/1 [==============================] - 0s 5ms/step
24: cow